# business Problem

#### Objetivo do Problema
    -- Previsão do primeiro destino que um novo usuario irá escolher.

#### Porque?
**Qual tipo de modelo de negócio do AirBNB?**
    
    -- MarketPlace (Conecta pessoas que oferecem acomodação, com pessoas que estão procurando acomodação).

##### Oferta (Pessoas oferecendo acomodação)

    -- Tamanho do portfolio
    -- Diversidade/densidade de portfolio
    -- Preço medio

##### Demanda (Pessoas procurando acomodação)

    -- Numero de usuarios
    -- LTV (Lifitime Value)
    -- CAC (Client Acquisition Cost)

**Cross Revenue** (FEE * Numero Cliente) - CAC

#### Proposta da Solução

    -- Modelo de predição do primeiro destino de um novo usuario.

##### Tipos de formatos para entrega do projeto.
    -- Predições salva em tabela do banco de dados.
    -- API: Input: Usuario e suas caracteristicas / Output: Usuario e suas caracteristicas com as Predições do destino.

# Ciclo03: Cross Validation

# 00 - Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import preprocessing as pp
from sklearn import metrics as m
from scikitplot import metrics as mt

from keras import models as ml
from keras import layers as l

2022-11-17 20:22:01.390236: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 20:22:01.473039: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-17 20:22:01.473051: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-17 20:22:01.491111: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-17 20:22:01.894302: W tensorflow/stream_executor/platform/de

## Paths

In [2]:
path_data = '/home/wellington/Documentos/Projetos/Projetos_ds_pa/airbnb_predict_first_booking/airbnb_predict_first_booking/data/'
path_model = '/home/wellington/Documentos/Projetos/Projetos_ds_pa/airbnb_predict_first_booking/airbnb_predict_first_booking/models/'
path_transformation = '/home/wellington/Documentos/Projetos/Projetos_ds_pa/airbnb_predict_first_booking/airbnb_predict_first_booking/transformation_data/'

## Loading Data

In [3]:
data = pd.read_csv(path_data+'train_users_2.csv')

In [4]:
data.head(1).T

,0
id,gxn3p5htnn
date_account_created,2010-06-28
timestamp_first_active,20090319043255
date_first_booking,NaN
gender,-unknown-
age,NaN
signup_method,facebook
signup_flow,0
language,en
affiliate_channel,direct


# 1.0 Data Description

## Data Dimensions

In [5]:
print("Number of Rows {}".format(data.shape[0]))
print("Number of Rows {}".format(data.shape[1]))

Number of Rows 213451
Number of Rows 16


## Data Types

In [6]:
data.dtypes

id                          object
date_account_created        object
timestamp_first_active       int64
date_first_booking          object
gender                      object
age                        float64
signup_method               object
signup_flow                  int64
language                    object
affiliate_channel           object
affiliate_provider          object
first_affiliate_tracked     object
signup_app                  object
first_device_type           object
first_browser               object
country_destination         object
dtype: object

## Check NA

In [7]:
data.isnull().sum()

id                              0
date_account_created            0
timestamp_first_active          0
date_first_booking         124543
gender                          0
age                         87990
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6065
signup_app                      0
first_device_type               0
first_browser                   0
country_destination             0
dtype: int64

### Drop NA

In [8]:
data = data.dropna()

In [9]:
data.isnull().sum()

id                         0
date_account_created       0
timestamp_first_active     0
date_first_booking         0
gender                     0
age                        0
signup_method              0
signup_flow                0
language                   0
affiliate_channel          0
affiliate_provider         0
first_affiliate_tracked    0
signup_app                 0
first_device_type          0
first_browser              0
country_destination        0
dtype: int64

## Change data type

In [10]:
#date_account_created 
data['date_account_created'] = pd.to_datetime( data['date_account_created'] )

#timestamp_first_active
data['timestamp_first_active'] = pd.to_datetime(data['timestamp_first_active'], format='%Y%m%d%H%M%S')

#date_first_booking
data['date_first_booking'] = pd.to_datetime(data['date_first_booking'])

#age
data['age'] = data['age'].astype(int)


In [11]:
data.dtypes

id                                 object
date_account_created       datetime64[ns]
timestamp_first_active     datetime64[ns]
date_first_booking         datetime64[ns]
gender                             object
age                                 int64
signup_method                      object
signup_flow                         int64
language                           object
affiliate_channel                  object
affiliate_provider                 object
first_affiliate_tracked            object
signup_app                         object
first_device_type                  object
first_browser                      object
country_destination                object
dtype: object

In [12]:
data.sample()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
189956,um5qewunq2,2014-05-15,2014-05-15 23:35:02,2014-12-16,FEMALE,32,facebook,0,en,sem-non-brand,google,omg,Web,Mac Desktop,Chrome,US


### Check Balanced Data

In [27]:
data['country_destination'].value_counts(normalize=True)

US       0.708864
other    0.110854
FR       0.054085
IT       0.029763
GB       0.025935
ES       0.024864
CA       0.015696
DE       0.012454
NL       0.008801
AU       0.006366
PT       0.002318
Name: country_destination, dtype: float64

Classe resposta bem desbalanceada, vamos seguir dessa forma para finalizarmos o pipeline com o primeiro ciclo de crisp

# 2.0 Feature Engineering

# 3.0 Data Filtering

In [14]:
data_filtering = data.copy()

# 4.0 Exploratory data analysis

# 5.0 Data Preparation 

In [15]:
data_preparation = data_filtering.copy()


### GetDummies values

In [16]:
#GetDummies values
data_preparation_dummies = pd.get_dummies(data_preparation.drop(['id', 'country_destination'], axis=1))

# Jooin id And country destination
data_preparation_final = pd.concat( [data_preparation[['id', 'country_destination']], data_preparation_dummies], axis=1)

# 6.0 Feature Selection

In [17]:
cols_drop = ['date_account_created', 'timestamp_first_active', 'date_first_booking']

data_selection = data_preparation_final.drop(cols_drop, axis=1)

In [18]:
X = data_selection.drop('country_destination', axis=1)
y = data_selection['country_destination'].copy()

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=32)

In [20]:
x_train = X_train.drop('id', axis=1)
x_val = X_val.drop('id', axis=1)

In [21]:
ohe = pp.OneHotEncoder()

y_train_nn = ohe.fit_transform( y_train.values.reshape( -1, 1)).toarray()

# 7.0 Machine Learning Model

## Cross-Validation

In [22]:
#Generate K-fold
num_folds = 5

balanced_acc_list = []
kappa_acc_list = []
i = 1 

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=32)

for train_ix, val_ix in kfold.split(x_train, y_train):
    print('Fold Number: {}/{}'.format(i, num_folds))


    #get fold
    x_train_folds = x_train.iloc[train_ix]
    y_train_folds = y_train.iloc[train_ix]

    x_val_fold = x_train.iloc[val_ix]
    y_val_folds = y_train.iloc[val_ix]

    #Target Hot-encoding
    ohe = pp.OneHotEncoder()

    y_train_fold_nn = ohe.fit_transform( y_train_folds.values.reshape( -1, 1)).toarray() #Precisamos transformar nosso y_train neste formato, pois rede neural so aceita dessa forma.

    #Model Definition
    model = ml.Sequential()

    model.add( l.Dense(256, input_dim=x_train.shape[1], activation= 'relu'))
    model.add( l.Dense( 11, activation = 'softmax'))

    #model Compile
    model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    #Train model
    model.fit( x_train_folds, y_train_fold_nn, epochs=100, batch_size=32, verbose=0)

    #Prediction
    pred_nn = model.predict(x_val_fold)

    #Invert prediction
    yhat_nn = ohe.inverse_transform(pred_nn)

    #Prediction prepare
    y_val_nn = y_val_folds.to_numpy()
    yhat_nn = yhat_nn.reshape(1, -1)[0]

    #Metrics
    
    ##Balanced Accuracy
    balanced_acc_nn = m.balanced_accuracy_score(y_val_nn, yhat_nn)
    balanced_acc_list.append(balanced_acc_nn)

    #Kappa Metric

    kappa_acc_nn = m.cohen_kappa_score(y_val_nn, yhat_nn)
    kappa_acc_list.append(kappa_acc_nn)

    i += 1

Fold Number: 1/5


2022-11-17 20:22:04.025660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 20:22:04.025812: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-17 20:22:04.025860: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-17 20:22:04.025900: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-11-17 20:22:04.025939: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

341/341 [==============================] - 0s 419us/step
Fold Number: 2/5
341/341 [==============================] - 0s 425us/step
Fold Number: 3/5
341/341 [==============================] - 0s 409us/step
Fold Number: 4/5
341/341 [==============================] - 0s 435us/step
Fold Number: 5/5
341/341 [==============================] - 0s 411us/step


In [26]:
print('Avg Balanced Accuracy: {} +/- {}'.format(np.round( np.mean(balanced_acc_list), 2), np.round(np.std(balanced_acc_list),4)))

print('Avg Kappa Accuracy: {} +/- {}'.format(np.round( np.mean(kappa_acc_list),4), np.round( np.std(kappa_acc_list),4)))

Avg Balanced Accuracy: 0.09 +/- 0.0004
Avg Kappa Accuracy: 0.0023 +/- 0.0012
